# Multi - Patient Script v1

## TDT Pipeline - Peter Yazdi

Welcome to the Multi Patient Script v1. This will be taking the TDT pipeline as of v1 and analyze the output folders of your choice (note will go through Multiple patient outputs). For this script to work accurately, please download correct dependencies and run whole script. If you want to analyze single patient, use the single patient script. 

For any concerns/help please email pyazdi@bbcrc.ca

## Imports 

In [77]:
import matplotlib.pyplot as plt
import SimpleITK as sitk
import nibabel as nib
import os
from json_minify import json_minify
import json
import numpy as np
from scipy.ndimage import zoom
import torch
from scipy.optimize import curve_fit

import matplotlib as mpl
import matplotlib.pyplot as plt
# ----------------------------
# Global styling (recommended)
# ----------------------------
mpl.rcParams.update({
    "font.size": 18,              # base size
    "font.weight": "bold",
    "axes.titleweight": "bold",
    "axes.titlesize": 22,
    "figure.titlesize": 26,
    "figure.titleweight": "bold",
    "axes.labelweight": "bold",
    "axes.labelsize": 20,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
})


classes = {
    "0": "background",
    "1": "body",
    "2": "kidney",
    "3": "liver",
    "4": "prostate",
    "5": "spleen",
    "6": "heart",
    "7": "salivary_glands"
}

## User adjustable parameters

In [78]:
current_folder = os.getcwd()
print("Current folder:", current_folder)
output_folder_prefix = "TDT_Output__"

output_folders = []
config_files = []
logging_files = []
ct_paths = []

for item in os.listdir(current_folder):
    if item.startswith(output_folder_prefix) and os.path.isdir(os.path.join(current_folder, item)):
        output_folders.append(os.path.join(current_folder, item))
output_folders.sort(key=lambda p: int(os.path.basename(p).split("CT_")[-1])) # sort in numerical order

num_patients = len(output_folders)
print(f"Number of patients found: {num_patients}")

print("Found output folders:")
for out in output_folders: 
    for item in os.listdir(os.path.join(current_folder, out)):
        if item.endswith(".json"):
            config_path = os.path.join(current_folder, out, item)
            config_files.append(config_path)
        if item.endswith(".log"):
            logging_path = os.path.join(current_folder, out, item)
            logging_files.append(logging_path)
        if item.startswith("PSMA"): # assuming CT files always start with PSMA
            ct_path = os.path.join(current_folder, out, item)
            ct_paths.append(ct_path)    
    
for i, out in enumerate(output_folders):
    print("Output folder:", out)
    print("  Config file:", config_files[i])
    print("  Logging file:", logging_files[i])
    print("  CT DICOM path:", ct_paths[i])



Current folder: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline
Number of patients found: 19
Found output folders:
Output folder: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1
  Config file: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/config_validation_study.json
  Logging file: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/logging_file_CT_1.log
  CT DICOM path: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/PSMA-01-002_PSMA-01-002_CT_2018-02-22_161707_._CT.WB.50cm_n335__00000
Output folder: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_2
  Config file: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_2/config_validation_study.json
  Logging file: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_2/logging_file_CT_2.log
  CT DICOM path: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_2/PSMA-01-054_PSMA-01-05

## CT Images 

In [79]:
show_plot = False # set to True to visualize CT mid slices
if show_plot:
    for i,ct in enumerate(ct_paths):
        # will plot 3 images : one for each axis
        figure, ax = plt.subplots(1,3, figsize= (20,5))
        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(ct)
        reader.SetFileNames(dicom_names)
        image = reader.Execute()
        array = sitk.GetArrayFromImage(image)[::-1]  # z, y, x
        figure.suptitle(f"Patient {i+1} CT mid slices", fontsize=20, fontweight='bold')
        ax[0].set_title('Axial slice', fontsize=18, fontweight='bold')
        ax[1].set_title('Coronal slice', fontsize=18, fontweight='bold')
        ax[2].set_title('Sagittal slice', fontsize=18, fontweight='bold')
        ax[0].imshow(array[array.shape[0]//2,:,:], cmap='gray',aspect='auto')
        ax[1].imshow(array[:,array.shape[1]//2,:], cmap='gray',aspect='auto')
        ax[2].imshow(array[:,:,array.shape[2]//2], cmap='gray',aspect='auto')
        ax[0].axis('off')
        ax[1].axis('off')
        ax[2].axis('off')
        plt.show()
    

## ATN Map and Activity Maps

In [80]:
shape = None     # set to something (non-None) if you want to skip computing
spacing = None   # set to something (non-None) if you want to skip computing

if shape is None or spacing is None:
    shape_list = []
    spacing_list = []

    for i, output_folder in enumerate(output_folders):
        with open(config_files[i], encoding="utf-8") as f:
            config = json.loads(json_minify(f.read()))

        # ---- config fields ----
        spect_subdir = config["subdir_names"]["spect_preprocessing"]
        sp = config["spect_preprocessing"]
        prefix = sp["name"]
        xy_dim = sp.get("xy_dim", None)  # could be None

        # ---- reference image path ----
        ref_img_path = os.path.join(
            output_folder,
            spect_subdir,
            f"{prefix}_tdt_roi_seg.nii.gz"
        )
        if not os.path.exists(ref_img_path):
            raise FileNotFoundError(f"Missing ref image: {ref_img_path}")

        ref_img = sitk.ReadImage(ref_img_path)

        # SITK size/spacing are in (x,y,z)
        size_xyz = np.array(ref_img.GetSize(), dtype=float)
        spacing_xyz = np.array(ref_img.GetSpacing(), dtype=float)  # typically mm

        # Convert size to array-order (z,y,x) like sitk.GetArrayFromImage would give
        shape_zyx = np.array([size_xyz[2], size_xyz[1], size_xyz[0]], dtype=float)

        # If no resize, scale=1
        if xy_dim is None:
            scale = 1.0
        else:
            scale = float(xy_dim) / float(shape_zyx[1])  # match your pipeline: scale based on Y

        # Post-zoom shape in (z,y,x)
        new_shape_zyx = tuple(np.round(shape_zyx * scale).astype(int).tolist())

        # Post-zoom spacing: spacing / scale
        new_spacing_xyz_mm = spacing_xyz / scale

        # Reorder to (z,y,x) and convert mm -> cm
        new_spacing_zyx_cm = tuple((new_spacing_xyz_mm[[2, 1, 0]] * 0.1).tolist())

        shape_list.append(new_shape_zyx)
        spacing_list.append(new_spacing_zyx_cm)

        print(
            f"{os.path.basename(output_folder)} | scale={scale:.4f} "
            f"| shape_zyx={new_shape_zyx} | spacing_zyx_cm={new_spacing_zyx_cm}"
        )

    if shape is None:
        shape = shape_list
    if spacing is None:
        spacing = spacing_list

print("Shape for all patients (z,y,x):", shape)
print("Spacing for all patients (cm, z,y,x):", spacing)


TDT_Output__CT_1 | scale=0.5000 | shape_zyx=(168, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.19531240463256838, 0.19531240463256838)
TDT_Output__CT_2 | scale=0.5000 | shape_zyx=(168, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.19531240463256838, 0.19531240463256838)
TDT_Output__CT_3 | scale=0.5000 | shape_zyx=(150, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.19531240463256838, 0.19531240463256838)
TDT_Output__CT_4 | scale=0.5000 | shape_zyx=(168, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.19531240463256838, 0.19531240463256838)
TDT_Output__CT_5 | scale=0.5000 | shape_zyx=(150, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.21484379768371584, 0.21484379768371584)
TDT_Output__CT_6 | scale=0.5000 | shape_zyx=(168, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.19531240463256838, 0.19531240463256838)
TDT_Output__CT_7 | scale=0.5000 | shape_zyx=(168, 256, 256) | spacing_zyx_cm=(0.6539999961853028, 0.19531240463256838, 0.19531240463256838)
TDT_Output__CT_8 | s

In [81]:
atn_map = []
act_map = {}

for i, output_folder in enumerate(output_folders):
    with open(config_files[i], encoding="utf-8") as f:
        config = json.loads(json_minify(f.read())) # Load configuration file
    
    subdir_name = config['subdir_names']
    frames = config['pbpk']['FrameStartTimes']
    
    spect_preprocessing_parameters = config['spect_preprocessing']
    pbpk_parameters = config['pbpk']
    
    atn_map_path = os.path.join(output_folder, subdir_name['spect_preprocessing'],f"{spect_preprocessing_parameters['name']}_atn_av.bin")
    atn_map.append(atn_map_path)
    
    act_map[i] = {}
    for f in frames:
        activity_map_path_single_frame = os.path.join(output_folder, subdir_name['pbpk'],f"{pbpk_parameters['name']}_{f}_act_av.bin")
        act_map[i][f] = activity_map_path_single_frame
    
    print(f"Patient {i+1}:")
    print("  Attenuation map path:", atn_map_path)
    print("  Exists:", os.path.exists(atn_map_path))
    print("Activity maps:")
    for f in frames:
        print(f"  Frame {f}:", act_map[i][f])
        print(f"    Exists: {os.path.exists(act_map[i][f])}")
        
    print("-----")
        
        


Patient 1:
  Attenuation map path: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/spect_preprocessing_outputs/spect_preprocessing_atn_av.bin
  Exists: True
Activity maps:
  Frame 120: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/pbpk_outputs/pbpk_120_act_av.bin
    Exists: True
  Frame 1440: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/pbpk_outputs/pbpk_1440_act_av.bin
    Exists: True
  Frame 2880: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/pbpk_outputs/pbpk_2880_act_av.bin
    Exists: True
  Frame 4620: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/pbpk_outputs/pbpk_4620_act_av.bin
    Exists: True
  Frame 9360: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_1/pbpk_outputs/pbpk_9360_act_av.bin
    Exists: True
-----
Patient 2:
  Attenuation map path: /home/jhubadmin/Theranostic-Virtual-Patient-Pipeline/TDT_Output__CT_2/spect_preprocessing_ou

In [82]:
show_plot = False # set to True to visualize attenuation and activity maps
if show_plot:
    for patient in range(num_patients):
        fig, ax = plt.subplots(1, 1 + len(frames), figsize=((1 + len(frames)) * 8, 8))
        ax = np.atleast_1d(ax)

        fig.suptitle(f"Patient {patient+1} Attenuation and Activity Maps",
                    fontsize=20, fontweight="bold")

        # --- Attenuation map ---
        atn_img = np.fromfile(atn_map[patient], dtype=np.float32).reshape(shape[patient])[::-1]
        ax[0].set_title("Attenuation map", fontsize=18, fontweight="bold")
        ax[0].imshow(atn_img[:, atn_img.shape[1] // 2, :], cmap="gray", aspect="auto")
        ax[0].axis("off")

        # --- One common scale for ALL activity frames (per patient) ---
        vmax = 0.0
        for f in frames:
            act_img = np.fromfile(act_map[patient][f], dtype=np.float32).reshape(shape[patient])[::-1]
            proj = np.max(act_img, axis=1)
            vmax = max(vmax, float(np.nanmax(proj)))

        vmin = 0.0
        if vmax <= vmin or not np.isfinite(vmax):
            vmax = vmin + 1e-6

        # --- Plot activity maps with shared vmin/vmax ---
        im = None
        for i, f in enumerate(frames):
            act_img = np.fromfile(act_map[patient][f], dtype=np.float32).reshape(shape[patient])[::-1]
            proj = np.max(act_img, axis=1)

            ax[i + 1].set_title(f"Activity map - Frame {f}", fontsize=18, fontweight="bold")
            im = ax[i + 1].imshow(proj, cmap="turbo", aspect="auto", vmin=vmin, vmax=vmax)
            ax[i + 1].axis("off")

        # --- One shared colorbar for all activity maps ---
        cbar = fig.colorbar(im, ax=ax[1:], fraction=0.046, pad=0.04)
        cbar.set_label("MBq/ml", fontsize=14, fontweight="bold")

        plt.show()


## Plot Recon SPECT images

In [83]:
# --- build paths ---
recon_spect_paths = {}
for i, output_folder in enumerate(output_folders):
    with open(config_files[i], encoding="utf-8") as f:
        config = json.loads(json_minify(f.read()))

    subdir_name = config["subdir_names"]
    frames_i = config["pbpk"]["FrameStartTimes"]
    sim_name = config["spect_simulation"]["name"]

    recon_spect_paths[i] = {}
    for f in frames_i:
        recon_spect_paths[i][f] = os.path.join(
            output_folder,
            subdir_name["spect_simulation"],
            f"{sim_name}_{f}min.nii" 
        )


In [84]:
show_plot = False  # set to True to visualize reconstructed SPECT images
if show_plot:
    # --- plot with shared colorbar per patient ---
    for key in recon_spect_paths.keys():
        frames_k = recon_spect_paths[key].keys()

        fig, ax = plt.subplots(1, len(frames_k), figsize=(len(frames_k) * 8, 8))
        ax = np.atleast_1d(ax)
        fig.suptitle(f"Patient {key+1} Reconstructed SPECT Image", fontsize=20, fontweight="bold")

        # compute shared vmax for this patient (shared scale across all frames)
        vmax = 0.0
        for f in frames_k:
            path = recon_spect_paths[key][f]
            if not os.path.exists(path):
                continue
            arr = sitk.GetArrayFromImage(sitk.ReadImage(path))[::-1]
            proj = np.max(arr, axis=1)
            vmax = max(vmax, float(np.nanmax(proj)))

        vmin = 0.0

        # plot
        im = None
        for indx, f in enumerate(frames_k):
            path = recon_spect_paths[key][f]

            if not os.path.exists(path):
                ax[indx].set_title(f"Frame {f} min (missing)", fontsize=14, fontweight="bold")
                ax[indx].axis("off")
                continue

            arr = sitk.GetArrayFromImage(sitk.ReadImage(path))[::-1]
            proj = np.max(arr, axis=1)

            ax[indx].set_title(f"Frame {f} min", fontsize=18, fontweight="bold")
            im = ax[indx].imshow(proj, cmap="turbo", aspect="auto", vmin=vmin, vmax=vmax)
            ax[indx].axis("off")

        # one shared colorbar for all frames
        if im is not None:
            cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            cbar.set_label("MBq/ml", fontsize=14, fontweight="bold") 


        plt.show()

## Comparison Study

### PBPK (Ground Truth) Set up

In [85]:
def load_tac_bins(roi, base_path, pbpk_prefix):
    roi = roi.lower()
    t  = np.fromfile(os.path.join(base_path, f"{pbpk_prefix}_{roi}_TAC_time.bin"), dtype=np.float32)
    act  = np.fromfile(os.path.join(base_path, f"{pbpk_prefix}_{roi}_TAC_values.bin"), dtype=np.float32)
    t_s = np.fromfile(os.path.join(base_path, f"{pbpk_prefix}_{roi}_sample_times.bin"), dtype=np.float32)
    act_s = np.fromfile(os.path.join(base_path, f"{pbpk_prefix}_{roi}_sample_values.bin"), dtype=np.float32)
    return t, act, t_s, act_s

pbpk_tacs = {}

for i, output_folder in enumerate(output_folders):
    with open(config_files[i], encoding="utf-8") as f:
        config = json.loads(json_minify(f.read()))

    pbpk_subdir = config["subdir_names"]["pbpk"]
    pbpk_params = config["pbpk"]
    pbpk_prefix = pbpk_params["name"]
    
    rois = config['spect_preprocessing']['roi_subset']

    base_path = os.path.join(output_folder, pbpk_subdir)

    pbpk_tacs[i] = {}   
    for roi in rois:
        t, act, t_s, act_s = load_tac_bins(roi, base_path, pbpk_prefix)
        pbpk_tacs[i][roi] = {
            "t": t,
            "act": act,
            "t_s": t_s,
            "act_s": act_s
        }


### Recon SPECT Setup 

#### Mask recon data

In [86]:
IDENTITY = (1,0,0, 0,1,0, 0,0,1)

def center_origin(img):
    size = np.array(img.GetSize(), dtype=float)    # (x,y,z)
    sp   = np.array(img.GetSpacing(), dtype=float) # (x,y,z)
    origin = -0.5 * (size - 1.0) * sp
    img.SetOrigin(tuple(origin.tolist()))
    img.SetDirection(IDENTITY)
    return img

def get_nii_data(nii_path, spacing_scale=1.0):
    img = sitk.ReadImage(nii_path)

    # scale spacing on the IMAGE (e.g., mm -> cm => 0.1) BEFORE centering
    sp = np.array(img.GetSpacing(), dtype=float) * float(spacing_scale)
    img.SetSpacing(tuple(sp))

    img = center_origin(img)
    arr_zyx = sitk.GetArrayFromImage(img)          # (z,y,x)
    spacing_xyz = img.GetSpacing()                 # (x,y,z)
    size_xyz = img.GetSize()                       # (x,y,z)
    vox_volume = spacing_xyz[0] * spacing_xyz[1] * spacing_xyz[2]
    return img, arr_zyx, spacing_xyz, size_xyz, vox_volume



In [87]:
seg_recons = []
logging_view = False # set to True to see resampling logs
plotting_view = False # set to True to see resampling plots

for i, output_folder in enumerate(output_folders):
    if i < 18:
        # Config stuff
        
        with open(config_files[i], encoding="utf-8") as f:
            config = json.loads(json_minify(f.read()))

        spect_preprocess_subdir = config["subdir_names"]["spect_preprocessing"]
        prefix_sp = config["spect_preprocessing"]["name"]

        spect_simulation_subdir = config["subdir_names"]["spect_simulation"]
        prefix_sim = config["spect_simulation"]["name"]

        frames = config["pbpk"]["FrameStartTimes"]
        ref_frame = frames[0]
        
        # Reference Images

        ref_seg_img_path = os.path.join(
            output_folder, spect_preprocess_subdir, f"{prefix_sp}_tdt_roi_seg.nii.gz"
        )
        ref_seg_img, ref_seg_arr, ref_seg_spacing_xyz, ref_seg_shape, _ = get_nii_data(
            ref_seg_img_path, spacing_scale=0.1
        )

        ref_recon_img_path = os.path.join(
            output_folder, spect_simulation_subdir, f"{prefix_sim}_{ref_frame}min.nii"
        )
        ref_recon_img, ref_recon_arr, ref_recon_spacing_xyz, ref_recon_shape, _ = get_nii_data(
            ref_recon_img_path, spacing_scale=1.0
        )
        
        # Resample seg to recon space
        seg_recon_img = sitk.Resample(ref_seg_img,             
                                    ref_recon_img,          
                                    sitk.Transform(),
                                    sitk.sitkNearestNeighbor,
                                    0,
                                    sitk.sitkInt16)
        seg_recon_arr = sitk.GetArrayFromImage(seg_recon_img)          # (z,y,x)
        seg_recons.append(seg_recon_img)
        
        
        if logging_view:
            print("\n")
            print(f"Processing patient {i+1} segmentation resampling...")
            
            print("SEG unique:", np.unique(ref_seg_arr))
            print("Ref seg shape:", ref_seg_shape,
                "spacing:", ref_seg_spacing_xyz,
                ref_seg_img.GetDirection(),
                ref_seg_img.GetOrigin())
            
            print("Ref recon shape:", ref_recon_shape,
                "spacing:", ref_recon_spacing_xyz,
                ref_recon_img.GetDirection(),
                ref_recon_img.GetOrigin())

            print("SEG->RECON unique:", np.unique(seg_recon_arr))
            print("seg recon shape:", seg_recon_arr.shape,
                "spacing:", seg_recon_img.GetSpacing(),
                seg_recon_img.GetDirection(),
                seg_recon_img.GetOrigin())
        
        if plotting_view:
            fig, ax = plt.subplots(1,2, figsize=(12,6))
            ax[0].set_title("Original Segmentation", fontsize=16, fontweight="bold")
            ax[0].imshow(np.max(ref_seg_arr, axis=1), cmap='gray',aspect='auto',origin='lower')
            ax[0].axis('off')
    
            ax[1].set_title("Resampled Segmentation + Recon", fontsize=16, fontweight="bold")
            ax[1].imshow(np.max(ref_recon_arr, axis=1), cmap='turbo',aspect='auto',origin='lower')
            ax[1].imshow(np.max(seg_recon_arr, axis=1), cmap='Reds',aspect='auto',origin='lower',alpha = 0.2)
            ax[1].axis('off')

            plt.suptitle(f"Patient {i+1} Segmentation Resampling", fontsize=20, fontweight="bold")
            plt.show()
        


In [106]:
def _build_class_map(arr, unique_values, classes):
        class_map = {}
        for value in unique_values:
            if value == 0:
                continue
            name = classes.get(str(value))
            if name is not None:
                class_map[name] = int(value)
        return class_map
masks_recon = {}  # {patient_index : {roi_name : mask_zyx_bool}}

for patient_idx, seg_recon_img in enumerate(seg_recons):
    seg_arr_zyx = sitk.GetArrayFromImage(seg_recon_img).astype(np.int16)

    class_map = _build_class_map(
        seg_arr_zyx,
        np.unique(seg_arr_zyx),
        classes
    )  # {roi_name: label_value}
    masks_recon[patient_idx] = {}

    for roi_name, label_val in class_map.items():
        mask_zyx = (seg_arr_zyx == label_val)   # bool mask, shape (z,y,x)
        masks_recon[patient_idx][roi_name] = mask_zyx
        


In [108]:
show_plot = False # set to True to see mask on recon image

if show_plot:
    for i, output_folder in enumerate(output_folders):
        if i <18:
            with open(config_files[i], encoding="utf-8") as f:
                    config = json.loads(json_minify(f.read()))

            # ---- config fields ----
            spect_simulation_subdir = config["subdir_names"]["spect_simulation"]
            sim = config["spect_simulation"]
            sp = config["spect_preprocessing"]
            prefix_sim = sim["name"]
            
            pbpk = config["pbpk"]
            frames = pbpk["FrameStartTimes"]
            ref_frame = frames[0] # just use first frame for reference
            
            roi_subset = sp["roi_subset"]
            roi_subset.append("body")
            
            
            # ---- reference images ----
            
            ref_recon_img_path = os.path.join(
                    output_folder,
                    spect_simulation_subdir,
                    f"{prefix_sim}_{ref_frame}min.nii"
                )
            
            ref_recon_img, ref_recon_arr, *_ = get_nii_data(ref_recon_img_path)
            
            # Plotting 
            figure, axes = plt.subplots(1, len(roi_subset), figsize=(10*len(roi_subset),10))
            
            for j,roi in enumerate(roi_subset):
                mask_zyx = masks_recon[i][roi]
                ax = axes[j]
                ax.set_title(f"Patient {i+1} – {roi} Mask", fontsize=18, fontweight='bold')
                ax.imshow(np.max(ref_recon_arr,axis = 1), cmap="turbo",origin = 'lower')
                ax.imshow(np.max(mask_zyx,axis = 1), cmap="gray", alpha=0.5, origin = 'lower')
                ax.axis("off")
            

            plt.show()
